In [94]:
import numpy as np
from sklearn.metrics import accuracy_score

In [95]:
num_classes = 10

In [96]:
base_predictions_path = f'./datasets/base_predictions/'
noisy_predictions_path = f'./datasets/noisy_predictions/'
results_path = f'./results/attack_rankings/'

In [97]:
# Choose attack methods
attacks = ["fgsm", "hopskip", "pgd"]

In [98]:
models = ["Resnet50","CNN","VGG","VGG_aug","Resnet101"]

In [99]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Test single file

In [146]:
# Load correct labels and predicted labels from .npy files
correct_labels = np.load('correct_labels.npy')
# predicted_labels = np.load('predicted_labels.npy')


In [155]:
predicted_labels = np.load('./datasets/noisy/hopskip/Resnet50.npy')

In [148]:
correct_labels = np.argmax(correct_labels, axis=1)

In [149]:
correct_labels.shape

(10000,)

In [156]:
# Calculate class-wise accuracy
class_accuracies = {}

for class_label in range(num_classes):
    # Identify indices where correct labels match the current class
    class_indices = correct_labels == class_label
    # print(type(class_indices))
    # Extract predicted labels corresponding to the current class
    class_predicted_labels = predicted_labels[class_indices]
    # print(correct_labels.shape)
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(correct_labels[class_indices], class_predicted_labels)
    
    class_accuracies[class_label] = class_accuracy

# Print class-wise accuracies
for class_label, class_accuracy in class_accuracies.items():
    print(f"Class {class_label} Accuracy: {class_accuracy * 100:.2f}%")

Class 0 Accuracy: 1.10%
Class 1 Accuracy: 0.10%
Class 2 Accuracy: 0.90%
Class 3 Accuracy: 86.70%
Class 4 Accuracy: 2.00%
Class 5 Accuracy: 0.30%
Class 6 Accuracy: 0.70%
Class 7 Accuracy: 0.00%
Class 8 Accuracy: 0.60%
Class 9 Accuracy: 0.70%


In [60]:
# Calculate overall accuracy
overall_accuracy = accuracy_score(correct_labels, predicted_labels)
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")

Overall Accuracy: 10.88%


In [61]:
print(f"Accuracy on Adversarial Examples: {overall_accuracy * 100:.2f}%")

Accuracy on Adversarial Examples: 10.88%


# Generate Accuracy Dict!

In [100]:
import os
import json

In [101]:
# Load correct labels and predicted labels from .npy files
correct_labels = np.load('correct_labels.npy')
correct_labels = np.argmax(correct_labels, axis=1)

In [102]:
accuracy = {}

In [103]:
noise_dir = "./datasets/noisy/"
base_dir = "./datasets/base/"

In [104]:
print(correct_labels.shape)

(10000,)


In [105]:
for i in range(num_classes):
    print(classes[i])
    accuracy[classes[i]] = {}
    for model in models:
        accuracy[classes[i]][model] = {}

        #base
        predictions_file = f'{base_dir}{model}.npy'
        if os.path.exists(predictions_file):
            predicted_labels = np.load(predictions_file)
            class_indices = correct_labels == i
            class_predicted_labels = predicted_labels[class_indices]
            base_accuracy = accuracy_score(correct_labels[class_indices], class_predicted_labels)
            accuracy[classes[i]][model]["base"] = base_accuracy
        else:
            print(f'The path {predictions_file} does not exist.')

        for attack in attacks:
            # Load predicted labels from numpy files
            predictions_file = f'{noise_dir}{attack}/{model}.npy'
            if not os.path.exists(predictions_file):
                print(f'The path {predictions_file} does not exist.')
                continue

            predicted_labels = np.load(predictions_file)
            class_indices = correct_labels == i
            class_predicted_labels = predicted_labels[class_indices]

            overall_accuracy = accuracy_score(correct_labels[class_indices], class_predicted_labels)
            accuracy[classes[i]][model][attack] = overall_accuracy

airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck


In [106]:
print(accuracy)

{'airplane': {'Resnet50': {'base': 0.939, 'fgsm': 0.781, 'hopskip': 0.011, 'pgd': 0.393}, 'CNN': {'base': 0.87, 'fgsm': 0.012, 'hopskip': 0.031, 'pgd': 0.197}, 'VGG': {'base': 0.776, 'fgsm': 0.132, 'hopskip': 0.087, 'pgd': 0.314}, 'VGG_aug': {'base': 0.849, 'fgsm': 0.0, 'hopskip': 0.039, 'pgd': 0.122}, 'Resnet101': {'base': 0.972, 'fgsm': 0.891, 'hopskip': 0.01, 'pgd': 0.695}}, 'automobile': {'Resnet50': {'base': 0.978, 'fgsm': 0.761, 'hopskip': 0.001, 'pgd': 0.385}, 'CNN': {'base': 0.966, 'fgsm': 0.016, 'hopskip': 0.015, 'pgd': 0.434}, 'VGG': {'base': 0.918, 'fgsm': 0.128, 'hopskip': 0.027, 'pgd': 0.567}, 'VGG_aug': {'base': 0.964, 'fgsm': 0.007, 'hopskip': 0.01, 'pgd': 0.307}, 'Resnet101': {'base': 0.975, 'fgsm': 0.82, 'hopskip': 0.001, 'pgd': 0.312}}, 'bird': {'Resnet50': {'base': 0.906, 'fgsm': 0.625, 'hopskip': 0.009, 'pgd': 0.306}, 'CNN': {'base': 0.834, 'fgsm': 0.067, 'hopskip': 0.072, 'pgd': 0.287}, 'VGG': {'base': 0.749, 'fgsm': 0.103, 'hopskip': 0.108, 'pgd': 0.349}, 'VGG_aug

In [107]:
with open('./results/accuracy.json', 'w') as json_file:
    json.dump(accuracy, json_file)

# Ranking

In [8]:
import os
import json

In [108]:
# Load the JSON data
with open('./results/accuracy.json', 'r') as file:
    data = json.load(file)

In [109]:
data

{'airplane': {'Resnet50': {'base': 0.939,
   'fgsm': 0.781,
   'hopskip': 0.011,
   'pgd': 0.393},
  'CNN': {'base': 0.87, 'fgsm': 0.012, 'hopskip': 0.031, 'pgd': 0.197},
  'VGG': {'base': 0.776, 'fgsm': 0.132, 'hopskip': 0.087, 'pgd': 0.314},
  'VGG_aug': {'base': 0.849, 'fgsm': 0.0, 'hopskip': 0.039, 'pgd': 0.122},
  'Resnet101': {'base': 0.972, 'fgsm': 0.891, 'hopskip': 0.01, 'pgd': 0.695}},
 'automobile': {'Resnet50': {'base': 0.978,
   'fgsm': 0.761,
   'hopskip': 0.001,
   'pgd': 0.385},
  'CNN': {'base': 0.966, 'fgsm': 0.016, 'hopskip': 0.015, 'pgd': 0.434},
  'VGG': {'base': 0.918, 'fgsm': 0.128, 'hopskip': 0.027, 'pgd': 0.567},
  'VGG_aug': {'base': 0.964, 'fgsm': 0.007, 'hopskip': 0.01, 'pgd': 0.307},
  'Resnet101': {'base': 0.975, 'fgsm': 0.82, 'hopskip': 0.001, 'pgd': 0.312}},
 'bird': {'Resnet50': {'base': 0.906,
   'fgsm': 0.625,
   'hopskip': 0.009,
   'pgd': 0.306},
  'CNN': {'base': 0.834, 'fgsm': 0.067, 'hopskip': 0.072, 'pgd': 0.287},
  'VGG': {'base': 0.749, 'fgsm':

In [110]:
# Define a function to calculate attack effectiveness
def calculate_effectiveness(base_accuracy, attack_accuracy):
    return base_accuracy - attack_accuracy

In [112]:
for class_name, class_data in data.items():
    print(class_name)
    # print(class_data)
    for model_name, model_data in class_data.items():
        print(model_name)
        print(model_data)
        for attack_name, attack_accuracy in model_data.items():
            if attack_name == "base":
                print(attack_name)
                print(attack_accuracy)
    break

airplane
Resnet50
{'base': 0.939, 'fgsm': 0.781, 'hopskip': 0.011, 'pgd': 0.393}
base
0.939
CNN
{'base': 0.87, 'fgsm': 0.012, 'hopskip': 0.031, 'pgd': 0.197}
base
0.87
VGG
{'base': 0.776, 'fgsm': 0.132, 'hopskip': 0.087, 'pgd': 0.314}
base
0.776
VGG_aug
{'base': 0.849, 'fgsm': 0.0, 'hopskip': 0.039, 'pgd': 0.122}
base
0.849
Resnet101
{'base': 0.972, 'fgsm': 0.891, 'hopskip': 0.01, 'pgd': 0.695}
base
0.972


In [113]:
# Initialize a dictionary to store results
attack_rankings = {}

for class_name, class_data in data.items():
    class_rankings = {}


    for attack in attacks:

        for model in models:

            if attack not in class_data[model]:
                continue
            effectiveness = calculate_effectiveness(class_data[model]['base'], class_data[model][attack])

            if attack not in class_rankings:
                class_rankings[attack] = effectiveness
            else:
                class_rankings[attack] += effectiveness

    print(class_rankings)

    # Rank the attacks for the current class
    ranked_attacks = sorted(class_rankings.items(), key=lambda x: x[1], reverse=True)

    # Store the ranked attacks in the overall dictionary
    attack_rankings[class_name] = ranked_attacks
    # break


{'fgsm': 2.5900000000000003, 'hopskip': 4.228, 'pgd': 2.685}
{'fgsm': 3.069, 'hopskip': 4.747, 'pgd': 2.7960000000000003}
{'fgsm': 2.5189999999999997, 'hopskip': 3.9619999999999997, 'pgd': 2.907}
{'fgsm': 2.3289999999999997, 'hopskip': 2.5620000000000003, 'pgd': 2.379}
{'fgsm': 2.585, 'hopskip': 4.218, 'pgd': 3.399}
{'fgsm': 2.952, 'hopskip': 3.891, 'pgd': 3.0250000000000004}
{'fgsm': 1.8499999999999999, 'hopskip': 4.643, 'pgd': 1.9459999999999997}
{'fgsm': 3.0060000000000002, 'hopskip': 3.5700000000000003, 'pgd': 3.08}
{'fgsm': 2.606, 'hopskip': 4.577999999999999, 'pgd': 2.3960000000000004}
{'fgsm': 2.933, 'hopskip': 4.25, 'pgd': 3.319}


In [114]:
# Print the results
for class_name, ranked_attacks in attack_rankings.items():
    print(f"Class: {class_name}")
    for rank, (attack_name, effectiveness) in enumerate(ranked_attacks, start=1):
        print(f"{rank}. {attack_name}: {effectiveness:.3f}")
    print("\n")

Class: airplane
1. hopskip: 4.228
2. pgd: 2.685
3. fgsm: 2.590


Class: automobile
1. hopskip: 4.747
2. fgsm: 3.069
3. pgd: 2.796


Class: bird
1. hopskip: 3.962
2. pgd: 2.907
3. fgsm: 2.519


Class: cat
1. hopskip: 2.562
2. pgd: 2.379
3. fgsm: 2.329


Class: deer
1. hopskip: 4.218
2. pgd: 3.399
3. fgsm: 2.585


Class: dog
1. hopskip: 3.891
2. pgd: 3.025
3. fgsm: 2.952


Class: frog
1. hopskip: 4.643
2. pgd: 1.946
3. fgsm: 1.850


Class: horse
1. hopskip: 3.570
2. pgd: 3.080
3. fgsm: 3.006


Class: ship
1. hopskip: 4.578
2. fgsm: 2.606
3. pgd: 2.396


Class: truck
1. hopskip: 4.250
2. pgd: 3.319
3. fgsm: 2.933




# Ranking one by one

In [189]:
# Load the JSON data
with open('./results/accuracy.json', 'r') as file:
    data = json.load(file)

In [190]:
attack_rankings = {}

In [191]:
def print_rankings():
    for class_name, ranked_attacks in attack_rankings.items():
        print(f"Class: {class_name}")
        for rank, (attack_name, effectiveness) in enumerate(ranked_attacks, start=1):
            print(f"{rank}. {attack_name}: {effectiveness:.3f}")
        print("\n")

In [192]:
def print_class_rankings(cls):
    for class_name, ranked_attacks in attack_rankings.items():
        if cls!=class_name:
            continue
        print(f"Class: {class_name}")
        for rank, (attack_name, effectiveness) in enumerate(ranked_attacks, start=1):
            print(f"{rank}. {attack_name}: {effectiveness:.3f}")
        print("\n")

In [193]:
def update_ranking(model_name):
    print(f'Updating rankings with model: {model_name}')
    for class_name, class_data in data.items():
        if class_name in attack_rankings:
            class_rankings = dict(attack_rankings[class_name])
        else:
            class_rankings = {}
        for attack in attacks:
            # for model in models:
            if model_name not in class_data:
                continue
            if attack not in class_data[model_name]:
                continue
            
            effectiveness = calculate_effectiveness(class_data[model_name]['base'], class_data[model_name][attack])

            if attack not in class_rankings:
                class_rankings[attack] = effectiveness
            else:
                class_rankings[attack] += effectiveness

        # Rank the attacks for the current class
        ranked_attacks = sorted(class_rankings.items(), key=lambda x: x[1], reverse=True)

        # Store the ranked attacks in the overall dictionary
        attack_rankings[class_name] = ranked_attacks
        # break
    # print_rankings()


In [194]:
print(attack_rankings)

{}


In [195]:
print_class_rankings("cat")

In [196]:
update_ranking(models[0])

Updating rankings with model: Resnet50


In [197]:
print_class_rankings("cat")

Class: cat
1. pgd: 0.258
2. fgsm: 0.228
3. hopskip: -0.003




In [198]:
update_ranking(models[1])

Updating rankings with model: CNN


In [199]:
print_class_rankings("cat")

Class: cat
1. fgsm: 0.925
2. pgd: 0.784
3. hopskip: 0.519




In [200]:
update_ranking(models[2])

Updating rankings with model: VGG


In [201]:
print_class_rankings("cat")

Class: cat
1. fgsm: 1.494
2. pgd: 1.191
3. hopskip: 1.008




In [202]:
update_ranking(models[3])

Updating rankings with model: VGG_aug


In [203]:
print_class_rankings("cat")

Class: cat
1. fgsm: 2.253
2. pgd: 1.812
3. hopskip: 1.685




In [204]:
update_ranking(models[4])

Updating rankings with model: Resnet101


In [205]:
print_class_rankings("cat")

Class: cat
1. hopskip: 2.562
2. pgd: 2.379
3. fgsm: 2.329




In [207]:
print_rankings()

Class: airplane
1. hopskip: 4.228
2. pgd: 2.685
3. fgsm: 2.590


Class: automobile
1. hopskip: 4.747
2. fgsm: 3.069
3. pgd: 2.796


Class: bird
1. hopskip: 3.962
2. pgd: 2.907
3. fgsm: 2.519


Class: cat
1. hopskip: 2.562
2. pgd: 2.379
3. fgsm: 2.329


Class: deer
1. hopskip: 4.218
2. pgd: 3.399
3. fgsm: 2.585


Class: dog
1. hopskip: 3.891
2. pgd: 3.025
3. fgsm: 2.952


Class: frog
1. hopskip: 4.643
2. pgd: 1.946
3. fgsm: 1.850


Class: horse
1. hopskip: 3.570
2. pgd: 3.080
3. fgsm: 3.006


Class: ship
1. hopskip: 4.578
2. fgsm: 2.606
3. pgd: 2.396


Class: truck
1. hopskip: 4.250
2. pgd: 3.319
3. fgsm: 2.933




In [145]:
with open('./results/final_rankings.json', 'w') as json_file:
    json.dump(attack_rankings, json_file)